In [13]:
from datetime import datetime
from dateutil import parser
import os
import json
import time
import gzip
import pandas as pd
import twython
from twython import TwythonRateLimitError
from twython import TwythonAuthError
import Twitter_scraping as scraper
import string
import pickle as pkl
import numpy as np
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError

import logging
logging.basicConfig(filename='Twitter.log',level=logging.DEBUG)

APP_KEY = 'ZIvxV0lZJnQI2FOuEqi0zzQqU'
APP_SECRET = '28z7HZuCNX71NMc5TO8ga3woravEt5wFsvm7Z2Q7LERBpoSCno'
twitter = twython.Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = twython.Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [14]:
f=open('followers_torinese_2.txt','r')
ID = [e.strip(' \n') for e in f.readlines()]
f.close()
date_start = parser.parse('2000-02-01 00:00:00+00:00')
already_scraped=[''.join([char for char in w if char.isdigit()]) for w in os.listdir('DF/USERS')]
ID=[i for i in ID if i not in already_scraped]
print(len(ID), 'users to scrape')

2302 users to scrape


In [21]:
def get_torinese_timeline(user_ID,date_start,twitter):
    #print('searching for user {}'.format(user_ID))
    searching = True
    last_id=None
    statuses=[]
    while (searching):
        time.sleep(.2)
        try:
            result = twitter.get_user_timeline(id = user_ID, count = 200, include_rts = True, trim_user=False, max_id=last_id)
        except (TwythonRateLimitError) as exc:
            print('Rate error')
            time.sleep(90)
            continue
        except TwythonAuthError:
            return None
        #except BaseException:
        #    print('Error. Sleeping 1 minute')
        #    time.sleep(60)
        #    continue

        if len(result) == 0:
            break
        # add the statuses to a list
        for status in result:
            created_at = parser.parse(status['created_at'])
            if created_at < date_start:
                # if created_at is less than date_start, stop searching
                searching = False
                break
            statuses.append(status)
            last_result = status
        # update max_id
        last_id = int(last_result['id_str']) - 1

    tweets = []

    for status in statuses:
        rt = 'retweeted_status' in status
        text = status['retweeted_status']['text'] if rt else status['text']
        
        for key in ['geo','coordinates','place']:
            if key not in status.keys():
                status[key]=''
        
        if 'location' not in status['user'].keys():
            status['user']['location']=''
        
        tweets.append((np.uint64(status['id_str']),
                       text,
                       rt,
                       status['created_at'],
                       status['lang'],status['geo'],status['coordinates'],status['place'],status['user']['location'],
                       np.uint64(status['user']['id_str']),status['user']['screen_name']))
    df = pd.DataFrame(tweets,
                  columns=['status_id', 'text', 'rt', 'created_at', 'lang', 'geo','coordinates','place','user_loc','user_id','user_name'])
    df = df.set_index('status_id')
    df.created_at = pd.to_datetime(df.created_at)
    with open('DF/USERS/timeline_user_{}.pkl'.format(user_ID), 'wb') as f:
        pkl.dump(statuses,f)
    return df

In [16]:
result = twitter.get_user_timeline(id = '1923655200', count = 1, include_rts = True, trim_user=False, max_id=None)
result[0]['user']

In [24]:
already_scraped=[''.join([char for char in w if char.isdigit()]) for w in os.listdir('DF/USERS')]
ID=[i for i in ID if i not in already_scraped]
print(len(ID), 'users to scrape')

2031 users to scrape


In [25]:
for user_ID in ID:
    print("scraping user {}".format(user_ID))
    while(True):
        try:
            df=get_torinese_timeline(user_ID,date_start,twitter)
        except (TwythonRateLimitError) as exc:
            print('Rate error')
            time.sleep(90)
            logging.exception("message")
            #The continue send us back to the beginning of the loop.
            continue
        except (TwythonAuthError) as exc:
            print('Auth error')
            logging.exception("message")
            #The continue send us back to the beginning of the loop.
            continue
        except ConnectionResetError:
            print('ConnectionResetError')
            logging.exception("message")
            continue
        if str(df)!='None':
            df.to_pickle('DF/USERS/dataframe_user_{}.pkl'.format(user_ID))
        break

scraping user 63733538
scraping user 602884373
scraping user 86725602
scraping user 946760707
scraping user 262739666
scraping user 2964090707
scraping user 1042677073
scraping user 723992002774360064
scraping user 961686187315548161
scraping user 864361525
scraping user 251033720
scraping user 286087497
scraping user 3078160936
scraping user 230221106
scraping user 214943254
scraping user 4715370923
scraping user 387659936
scraping user 185562991
scraping user 439565702
scraping user 1048165406
scraping user 4873933121
scraping user 47091999
scraping user 1201644380
scraping user 447855131
scraping user 200795283
scraping user 2828191516
scraping user 15106471
scraping user 31486681
scraping user 415520888
scraping user 404189855
scraping user 236589903
scraping user 144859478
scraping user 221323177
scraping user 405937501
scraping user 216693159
scraping user 2160056493
scraping user 103333952
scraping user 281753253
scraping user 2352872256
scraping user 202123040
scraping user 891

scraping user 82386888
scraping user 102097520
scraping user 723426049627852800
scraping user 861327685
scraping user 1448713544
scraping user 2964119536
scraping user 71547795
scraping user 2281503736
scraping user 186460389
scraping user 1030453633
scraping user 775249411022684161
scraping user 828433640
scraping user 3935988802
scraping user 38706653
scraping user 2567563814
scraping user 214067583
scraping user 1057941979
scraping user 320287268
scraping user 2540842325
scraping user 803619135393447936
scraping user 1377137856
scraping user 1220058546
scraping user 516885361
scraping user 577520056
scraping user 283244749
scraping user 3247761185
scraping user 466419130
scraping user 927576552363216898
scraping user 1004850900
scraping user 3993800979
scraping user 494366532
scraping user 636447190
scraping user 1325004632
scraping user 902206158
scraping user 369284894
scraping user 1100613626
scraping user 546866930
scraping user 462787009
scraping user 43193096
scraping user 863

scraping user 584152141
scraping user 305078988
scraping user 462051709
scraping user 600281391
scraping user 149113305
scraping user 349154578
scraping user 1876772558
scraping user 348120484
scraping user 88949673
scraping user 49129760
scraping user 2580246372
scraping user 301356179
scraping user 834028297
scraping user 2810792277
scraping user 105521654
scraping user 194602976
scraping user 41422379
scraping user 838736780199354369
scraping user 914306294
scraping user 492651245
scraping user 199393094
scraping user 265067104
scraping user 1099121976
scraping user 302057266
scraping user 501897447
scraping user 520407282
scraping user 2277207951
scraping user 962751112691552256
scraping user 583764156
scraping user 809510572773543936
scraping user 436828950
scraping user 1373272969
scraping user 579187212
scraping user 551735242
scraping user 2152768895
scraping user 4778133497
scraping user 30068247
scraping user 73334841
scraping user 27214614
scraping user 1118248327
scraping u

scraping user 1151614388
scraping user 958802833788334088
scraping user 300190343
scraping user 767360216258191360
scraping user 65111941
scraping user 834094134692552704
scraping user 895409017
scraping user 301130381
scraping user 846646447487488002
scraping user 266826941
scraping user 1528184425
scraping user 1538099952
scraping user 63658443
scraping user 856440388688785408
scraping user 1246200276
scraping user 186816752
scraping user 529524995
scraping user 382451286
scraping user 813863431422046208
scraping user 508865507
scraping user 17210771
scraping user 1264233007
scraping user 281448987
scraping user 2688651677
scraping user 1337282858
scraping user 2216037298
scraping user 391445592
scraping user 1871098567
scraping user 538563605
scraping user 387973281
scraping user 576301823
scraping user 712925572
scraping user 1050911497
scraping user 1307095242
scraping user 346666412
scraping user 397232618
scraping user 312147793
scraping user 717973335351623680
scraping user 711

scraping user 3816960495
scraping user 245367472
scraping user 634888297
scraping user 522118512
scraping user 430599737
scraping user 139349705
scraping user 2394036643
scraping user 549532714
scraping user 605349143
scraping user 41561390
scraping user 789297110
scraping user 1089611533
scraping user 876736730489118720
scraping user 391648923
scraping user 235993844
scraping user 239724198
scraping user 1072294190757933056
scraping user 327366817
Rate error
scraping user 337866561
scraping user 985079785876606976
scraping user 1725145796
scraping user 108274998
scraping user 410625892
scraping user 489776725
scraping user 73959179
scraping user 1104238423
scraping user 1468192572
scraping user 761672039551893504
scraping user 1669057508
scraping user 726904560
scraping user 325519183
scraping user 224693685
scraping user 348497217
scraping user 797145207955398656
scraping user 3917771475
scraping user 41781716
scraping user 837045010986053632
scraping user 2890607727
scraping user 49

scraping user 857300386411098112
scraping user 2363590581
scraping user 334583796
scraping user 33939963
scraping user 1006097617238011905
scraping user 518388529
scraping user 1443848851
scraping user 2212291501
scraping user 930351412973899776
scraping user 1427273120
scraping user 243227180
scraping user 3303588621
scraping user 291017749
scraping user 1157242675
scraping user 976601125
scraping user 264306396
scraping user 517699216
scraping user 3614858957
scraping user 1469872716
scraping user 369445205
scraping user 414765712
scraping user 376441298
scraping user 1536114373
scraping user 472910529
scraping user 3344139526
scraping user 3307875581
scraping user 2847899541
scraping user 710529645356171266
scraping user 33005730
scraping user 103065652
scraping user 543216877
scraping user 3200894007
scraping user 29480944
Rate error
scraping user 344709270
scraping user 1916455224
scraping user 176126741
scraping user 1033384850
scraping user 2348951269
scraping user 922027182
scr

scraping user 436558234
scraping user 3003494188
scraping user 578179559
scraping user 12634312
scraping user 615032747
scraping user 923663983
scraping user 370209169
scraping user 104187272
scraping user 1029702901852262401
scraping user 317808743
scraping user 408796442
scraping user 300514327
scraping user 402677679
scraping user 431972076
scraping user 572041634
scraping user 804249349656113152
scraping user 115959836
scraping user 182915556
scraping user 275966374
scraping user 23212280
Rate error
scraping user 96952326
scraping user 2430814133
scraping user 313444604
scraping user 823205978
scraping user 430549523
scraping user 166118537
scraping user 4832534800
scraping user 467144026
scraping user 1040118710
scraping user 462228039
scraping user 396890234
scraping user 739927845351362560
scraping user 867306553254309888
scraping user 226271366
scraping user 109620460
scraping user 1062883610
scraping user 1050448521093099523
scraping user 565942828
scraping user 44336188
scrap

In [59]:
ct_coordinates=0
ct_place=0
ct_uloc=0
ct_error=0
Locs={}
for uid in os.listdir('DF/USERS/'):
    df_u=pd.read_pickle('DF/USERS/'+uid)
    try:
        if set(df_u.user_loc)!={''} and set(df_u.user_loc)!=set():
            for loc in set(df_u.user_loc):
                try:
                    Locs[loc]+=1
                except KeyError:
                    Locs[loc]=1
            ct_uloc+=1
        ct_coordinates+=df_u[df_u.fillna('').coordinates!=''].shape[0]
        ct_place+=df_u[df_u.fillna('').place!=''].shape[0]  
    except AttributeError:
        ct_error+=1
        pass
ct_addresses=0
for k in Locs.keys():
    if any(char.isdigit() for char in k):
        ct_addresses+=Locs[k]
        print(k, Locs[k])
print (ct_addresses)

In [70]:
ct_coordinates,ct_place,ct_uloc,ct_error,ct_addresses

(208419, 390905, 7906, 344, 341)

In [65]:
 # number of users that gave their full adress

Torino, Piemonte via dei gladioli 39 1
Torino Via Rossini n.°6 1
Torino, via Maria Vittoria 20 1
Corso Moncalieri, 18 - Torino 1
Torino - via massena 26 1
spazio211 Torino 1
Via Torino, 23 Volpiano (TO) 1
Via Montevideo 27/P Torino,  1
Via Rossini 20,Torino 1
Via Luserna di Rorà, 8 Torino 1
Torino corso Sebastopoli 272/e 1
Via Po, 55 Torino 1
Via Pigafetta 3, Torino 1
viale Virgilio 25 Torino 1
Corso Francia 273, Torino (TO) 1
 Via Savigliano, 5 Torino 1
Via Montecuccoli 9, Torino. 1
Via Nizza 28 Torino, Italy 1
Via Pittara, 34 - Torino 1
Via  san secondo 15 Torino  1
Via C. G. Brugnone 12, Torino 1
via Giolitti 18, 10123 Torino 1
Via Stampatori, 4, Torino 1
Via San Secondo 46, Torino 1
Corso Trento 13 - Torino 1
Corso Moncalieri, 190 - Torino 1
Torino, Piemonte - Piazza della Repubblica, 1bis - 10122 1
Torino,corso Mediterraneo,84 1
Via Mazzini 33, Torino - ITALY 1
Via Murazzi Del Po, 26 -Torino 1
via Carlo Alberto 18 Torino 1
Via Paolo Sarpi, 111 Torino 1
Via Torino, 36 Poirino (Tori